# Rheostat for Collaboration
In this notebook, I will outline a possible model that can be used in conjunction with others.<br>
7.10.2020<br>
Ankita Roychoudhury

In [1]:
# bioscrape
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pandas as pd

from scipy.integrate import simps
from numpy import trapz

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
bokeh.io.output_notebook()

# from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
# from biocrnpyler import Mixture
# from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
from biocrnpyler import *

Loading BokehJS ...

In [2]:
from biocrnpyler.components_basic import MultiEnzyme


## Define Custom Mechanism, Component, and Mixture
The Mechanism was implemented in BioCRNPyler as MichalisMentenReversible and the Component was implemented as MultiEnzyme. My code has minor adjustments (related to parameters, reversibility, and 

MichalisMentenReversible(Mechanism): <br>
MultiEnzyme(Component):
    def __init__(self, enzyme, substrates, products, **keywords):
    

In [3]:
# Define some rate parameters
k_bf = 15
k_uf = 15

# Define all enzymes


In [4]:
class EnergyTxTl(Mixture):
    def __init__(self, name="",**keywords): 
        

        mech_cat = MichalisMentenReversible('catalysis')
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        #species = [self.enzyme]
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords) 
        #  parameter_file = 'parameters.txt'

In [5]:
def plot_entire_with_enzymes(k_bf, k_uf, k_cat_atp, enz_init, glucose_init, atp_init, pi_init, nadp_init, atpase_init, 
                             t_max, parameters,bpg_13 = 0,
                            area_isobss = False):

    k_bf = k_bf #20 #660
    k_uf = k_uf #20 #144 # per hour
    

    #

    E1_hex = MultiEnzyme(enzyme = "hex", substrates = ['glucose','atp'],
                         products = ['g6p','adp'], parameters = parameters)

    E2_pgi = MultiEnzyme(enzyme = 'pgi', substrates = ['g6p'],
               products = ['f6p'], kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E3_pfk = MultiEnzyme(enzyme = 'pfk', substrates = ['f6p','atp'], 
                         products = ['f16p','adp'], kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E4_ald_tpi = MultiEnzyme(enzyme ='ald_tpi' , substrates = ['f16p'], products = ['g3p', 'g3p'], 
                 kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E5_gapN = MultiEnzyme(enzyme ='gapN' , substrates = ['g3p', 'g3p','nadp', 'nadp'], 
                          products = ['a3pg', 'a3pg','nadph', 'nadph'], kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E6_mGapDH = MultiEnzyme(enzyme ='mGapDH' , substrates = ['g3p', 'g3p','pi', 'nadp', 'nadp'], 
                            products = ['a13bpg','nadph', 'nadph'],kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E7_pgk = MultiEnzyme(enzyme = 'pgk', substrates = ['a13bpg','adp'], products = ['a3pg', 'a3pg','atp'], 
                         kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E8_pgm = MultiEnzyme(enzyme ='pgm' , substrates = ['a3pg', 'a3pg'], products = ['a2pg', 'a2pg'],kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E9_eno = MultiEnzyme(enzyme ='eno' , substrates = ['a2pg', 'a2pg'], products = ['pep', 'pep'], 
            kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E10_pyk = MultiEnzyme(enzyme = 'pyk', substrates = ['pep', 'pep', 'adp', 'adp'],
                          products = ['pyruvate', 'pyruvate', 'atp', 'atp'], kb = k_bf, ku = 0.1*k_bf, parameters = parameters) # irreversible

    E11_alsS = MultiEnzyme(enzyme = 'alsS', substrates = ['pyruvate', 'pyruvate'], products = ['acetolac'], 
         kb = k_bf, ku = 0.1*k_bf, parameters = parameters) # irreversible

    E12_IlvC = MultiEnzyme(enzyme = 'IlvC', substrates = ['acetolac','nadph'], products = ['a23dih3mebut','nadp'], 
                           kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E13_IlvD = MultiEnzyme(enzyme ='IlvD' , substrates = ['a23dih3mebut'], products = ['a3me2oxo'], 
                kb = k_bf, ku = 0.1*k_bf, parameters = parameters)

    E14_kivD = MultiEnzyme(enzyme ='kivD' , substrates = ['a3me2oxo'],products = ['isobutanal'],
                           kb = k_bf, ku = 0.1*k_bf, parameters = parameters) # irreversible

    E15_yahk = MultiEnzyme(enzyme = 'yahk', substrates = ['isobutanal','nadph'], products = ['isobutanol','nadp'], kb = k_bf, ku = 0.1*k_bf, parameters = parameters)
    
    E16_atpase = MultiEnzyme(enzyme = 'atpase',  products = ['atp'],
                        substrates = ['adp', 'pi'], kb = k_bf, ku = 0.1*k_bf, parameters = parameters)


#     E16_all_other_atp = MultiEnzyme(enzyme = 'atp_synthase', substrates = [], fuel = ['atp'], 
#                                products = [], waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, k_cat = 1,)

    myMixture = EnergyTxTl(components = [E1_hex,E2_pgi,E3_pfk, E4_ald_tpi, E5_gapN, E6_mGapDH, E7_pgk, E8_pgm, E9_eno, E10_pyk, 
                                        E11_alsS, E12_IlvC, E13_IlvD, E14_kivD, E15_yahk, E16_atpase])
    myMixture_atp = EnergyTxTl(components = [E16_atpase])
    
    CRN = myMixture.compile_crn()
    CRN_atp = myMixture_atp.compile_crn()


    
    CRN.write_sbml_file("CRN.sbml")
    # CRN_atp.write_sbml_file("CRN_atp.sbml")
    
    timepoints = np.linspace(0,t_max,t_max)
    
    e = enz_init #0.15
    #e4 = 3.5
    atp = atp_init #30
    pi = pi_init
    atpase = atpase_init
    x0 = {'glucose':glucose_init, #30,
          'atp': atp,
         'nadp':nadp_init,
          'pi':pi,
          'a13bpg':bpg_13,
          "protein_hex":e,
          'protein_pgi':e,
          'protein_pfk':e,
          'protein_ald_tpi':e,
          'protein_gapN':e, 
          'protein_mGapDH':e,
          'protein_pgk':e,
          'protein_pgm':e,
          'protein_eno':e,
          'protein_pyk':e,
          'protein_alsS':e,
          'protein_IlvC':e,
          'protein_IlvD':e,
          'protein_kivD':e,
          'protein_yahk':e,
          'protein_atpase':atpase
        #  "protein_atp_synthase":e4
    }
    x0_atp = {'atp': atp,
               'protein_atpase':atpase
        #  "protein_atp_synthase":e4
    }

    re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
    re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

        # same color scheme as the others for presentation
    colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']
    #colors = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

    # First plot glucose and isobutanol
    p3_complex = bokeh.plotting.figure(width = 450, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration (mM)',
                             title = 'Entire Pathway Biocrnpyler')
    p3_complex.line(timepoints, re['glucose'], color = colors[0],line_width = 2, legend_label = 'glucose')
    p3_complex.line(timepoints, re['isobutanol'], color = colors[1], line_width = 2,legend_label = 'isobutanol')
    p3_complex.line(timepoints, re['f16p'], color = colors[2], line_width = 2,legend_label = 'f16p')
    p3_complex.legend.location = 'center_right'
    p3_complex.legend.click_policy="hide"


    # Plot ATP and ADP, Pi
    p4_complex = bokeh.plotting.figure(width = 450, height = 250,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                                      title = 'ATP Compare')
#     p4_complex.line(timepoints, re['molecule_glucose'], color = colors[0],# 0
#                     line_width = 1.5, legend_label = 'glucose')


    p4_complex.line(timepoints, re['atp'], color = colors[3], line_width = 1.5,legend_label = 'atp')
    p4_complex.line(timepoints, re_atp['atp'], legend_label = 'atp only',line_width=2, color = colors[4])


    p4_complex.line(timepoints, re['pi'], color = 'bisque',line_width = 2, legend_label = 'pi')#colors[6]
    p4_complex.line(timepoints, re['adp'], color = 'burlywood', line_width = 2,legend_label = 'adp')#colors[5]
    p4_complex.legend.location = 'center_right'
    p4_complex.legend.click_policy="hide"
    
    p5_enzyme = bokeh.plotting.figure(width = 650, height = 500,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                                     title = 'All Enzymes')
    
    enz_color = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']
    enz_color2 = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']
    p5_enzyme.line(timepoints, re['protein_hex'], color = enz_color[0],line_width = 2, legend_label = 'hex')
    p5_enzyme.line(timepoints, re['protein_pgi'], color = enz_color[1],line_width = 2, legend_label = 'pgi')
    p5_enzyme.line(timepoints, re['protein_pfk'], color = enz_color[2],line_width = 2, legend_label = 'pfk')
    p5_enzyme.line(timepoints, re['protein_ald_tpi'], color = enz_color[3],line_width = 2, legend_label = 'ald_tpi')
    p5_enzyme.line(timepoints, re['protein_gapN'], color = enz_color[4],line_width = 2, legend_label = 'gapN')
    p5_enzyme.line(timepoints, re['protein_mGapDH'], color = enz_color[5],line_width = 2, legend_label = 'mgapdh')
    p5_enzyme.line(timepoints, re['protein_pgk'], color = enz_color[6],line_width = 2, legend_label = 'pgk')
    p5_enzyme.line(timepoints, re['protein_pgm'], color = enz_color[7],line_width = 2, legend_label = 'pgm')
    p5_enzyme.line(timepoints, re['protein_eno'], color = enz_color[8],line_width = 2, legend_label = 'eno')
    p5_enzyme.line(timepoints, re['protein_pyk'], color = enz_color[9],line_width = 2, legend_label = 'pyk')
    p5_enzyme.line(timepoints, re['protein_alsS'], color = enz_color[10],line_width = 2, legend_label = 'alsS')
    p5_enzyme.line(timepoints, re['protein_IlvC'], color = enz_color[11],line_width = 2, legend_label = 'IlvC')
    p5_enzyme.line(timepoints, re['protein_IlvD'], color = enz_color2[0],line_width = 2, legend_label = 'IlvD')
    p5_enzyme.line(timepoints, re['protein_kivD'], color = enz_color2[1],line_width = 2, legend_label = 'kivD')
    p5_enzyme.line(timepoints, re['protein_yahk'], color = enz_color2[2],line_width = 2, legend_label = 'yahk')
    p5_enzyme.line(timepoints, re['protein_atpase'], color = enz_color2[3],line_width = 2, legend_label = 'atpase from rheostat + leak')
    p5_enzyme.line(timepoints, re_atp['protein_atpase'], color = enz_color2[4],line_width = 2, legend_label = 'atpase from leak only')

    p5_enzyme.legend.click_policy="hide"

    if area_isobss == False:
    
        # Find area under atp curve
        y = re['atp'].values
        dx = timepoints[1] - timepoints[0]
        area1 = trapz(y, dx=dx)
        print('area atp pink',area1)

        # Find area under atp only curve
        y2 = re_atp['atp'].values
        area2 = trapz(y2, dx=dx)
        print('area atp only green', area2)
        #print("area2", area2)


        #bokeh.io.show(row(p3_complex,p4_complex))
        return p3_complex,p4_complex, p5_enzyme,re
        #return area1
        
    elif area_isobss == True:
        y = re['atp'].values
        dx = timepoints[1] - timepoints[0]
        area1 = trapz(y, dx=dx)
        
        isob_ss = re['isobutanol'][t_max]
        
        return area1, isob_ss
        

In [6]:
parameters = {("catalysis", "kb1"):15,
             ("catalysis", "kb2"):15,
             ("catalysis", "ku1"):1.5,
              ("catalysis", "ku2"):1.5,
              ("catalysis",'atpase', "kcat"):2.0,
             ("catalysis", "kcat"):36000,
             ("catalysis", "kcat_rev"):0.00001} #Parameters in the parameter file can be overwritten by passing in a parameter dictionary


x1,x2,x_enz,df3 = plot_entire_with_enzymes(k_bf = 15, k_uf = 15, k_cat_atp = 2.0, enz_init = 0.2, glucose_init = 30,
            atp_init = 30, pi_init = 30, nadp_init = 30, atpase_init = 2., t_max = 72,parameters = parameters,
                                          area_isobss = False)
bokeh.io.show(row(x1,x2)) 
bokeh.io.show(x_enz)

ValueError: parameter_key must be None, a dictionary, a ParameterKey, a 3-tuple, or a string (parameter name): received ('catalysis', 'kb1').